# Script 01: Process CESM output for figures
Rich Fiorella, 3/11/2021.

This script reads in iCAM5, iCAM6 (nudged and free) data and pre-processes it to create standardized output files that can be used to run the scripts downstream.

The remaining scripts (02 - isotope figures/comparison of CAM5 and CAM6; 03 - figures relating to testing rainout hypotheses; 04 - figures relating to residence time; 05 - figures related to d-excess variability testing; 06 - correlation time dependence (daily, monthly vs interannual)).

In [2]:
# set up packages and data
from datetime import timedelta
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.gridspec as gridspec
import numpy as np
import pandas as pd
import xarray as xr
import cartopy.crs as ccrs
import cartopy
from cftime import DatetimeNoLeap
import h5py


# calculate d18O_0, need to filter for things above x mm/day
thres = 0.1 # mm/day 
thres_mps = thres*(1/86400)*(1/1000) # convert mm/day -> m/s

In [3]:
# Process iCAM6 - free for seasonal averages

ds_free = xr.open_dataset("..//raw_data/ts_free_subset_2x2.nc", decode_timedelta = False).drop('time_bnds')
ds_free
#===============================
#-------------------------------
# work through free dataset.

# filter out values below thres_mps?
ds_free['PRECT'] = ds_free['PRECT'].where(ds_free.PRECT > thres_mps, thres_mps)

# PRECTtime between 0 and 60
ds_free['PRECTtime'] = ds_free['PRECTtime'].where(ds_free.PRECTtime < 40, 40)
ds_free['PRECTtime'] = ds_free['PRECTtime'].where(ds_free.PRECTtime > 0, 0)

# d180 between -70 and +10
ds_free['PRECT_d18O'] = ds_free['PRECT_d18O'].where(ds_free.PRECT_d18O > -70, -70)
ds_free['PRECT_d18O'] = ds_free['PRECT_d18O'].where(ds_free.PRECT_d18O < 10, 10)

# d-excess between -50 and 50
ds_free['PRECT_dxs'] = ds_free['PRECT_dxs'].where(ds_free.PRECT_dxs > -25, -25)
ds_free['PRECT_dxs'] = ds_free['PRECT_dxs'].where(ds_free.PRECT_dxs < 50, 50)

# take seasonally weighted averages
#---------------------------------
# shift to get correct months! (UGHHHHH)
# for some reason, xarray reads in the end of the averaging time rather
# than the midpoint of the averaged month (UGH)
# fix from here: https://bb.cgd.ucar.edu/cesm/threads/external-tools-dont-like-cesm-time-coordinate.4604/
time_index_shifted = ds_free.time.get_index('time') - timedelta(days=1) 
ds_free['time'] = time_index_shifted

# calculate weighted mean using code from xarray tutorial:
# http://xarray.pydata.org/en/stable/examples/monthly-means.html
month_length = ds_free.time.dt.days_in_month
# Calculate the weights by grouping by 'time.season'.
weights = month_length.groupby('time.season') / month_length.groupby('time.season').sum()

# Test that the sum of the weights for each season is 1.0
np.testing.assert_allclose(weights.groupby('time.season').sum().values, np.ones(4))

# Calculate the weighted average
ds_free_weighted = (ds_free * weights).groupby('time.season').sum(dim='time')

# write out files.
ds_free.to_netcdf('../proc_data/iCAM6_free_monthly.nc')
ds_free_weighted.sel(season='JJA').to_netcdf('../proc_data/iCAM6_free_JJA.nc')
ds_free_weighted.sel(season='DJF').to_netcdf('../proc_data/iCAM6_free_DJF.nc')
ds_free.mean(dim="time").to_netcdf('../proc_data/iCAM6_free_ANN.nc')

ds_free

del ds_free

/Users/rfiorella/opt/anaconda3/envs/xarray/lib/python3.9/site-packages/xarray/coding/variables.py:173: RuntimeWarning: invalid value encountered in cast
  encoding["_FillValue"] = dtype.type(fv)
/Users/rfiorella/opt/anaconda3/envs/xarray/lib/python3.9/site-packages/xarray/coding/variables.py:180: RuntimeWarning: invalid value encountered in cast
  encoding["missing_value"] = dtype.type(mv)
/Users/rfiorella/opt/anaconda3/envs/xarray/lib/python3.9/site-packages/xarray/coding/variables.py:173: RuntimeWarning: invalid value encountered in cast
  encoding["_FillValue"] = dtype.type(fv)
/Users/rfiorella/opt/anaconda3/envs/xarray/lib/python3.9/site-packages/xarray/coding/variables.py:180: RuntimeWarning: invalid value encountered in cast
  encoding["missing_value"] = dtype.type(mv)
/Users/rfiorella/opt/anaconda3/envs/xarray/lib/python3.9/site-packages/xarray/coding/variables.py:173: RuntimeWarning: invalid value encountered in cast
  encoding["_FillValue"] = dtype.type(fv)
/Users/rfiorella/op

In [4]:
# Process iCAM6 - nudged for seasonal averages

ds_nudg = xr.open_dataset("../raw_data/ts_nudg_subset_2x2.nc", decode_timedelta = False).drop('time_bnds')

# filter out values below thres_mps?
ds_nudg['PRECT'] = ds_nudg['PRECT'].where(ds_nudg.PRECT > thres_mps, thres_mps)

# PRECTtime between 0 and 60
ds_nudg['PRECTtime'] = ds_nudg['PRECTtime'].where(ds_nudg.PRECTtime < 40, 40)
ds_nudg['PRECTtime'] = ds_nudg['PRECTtime'].where(ds_nudg.PRECTtime > 0, 0)

# d180 between -70 and +10
ds_nudg['PRECT_d18O'] = ds_nudg['PRECT_d18O'].where(ds_nudg.PRECT_d18O > -70, -70)
ds_nudg['PRECT_d18O'] = ds_nudg['PRECT_d18O'].where(ds_nudg.PRECT_d18O < 10, 10)

# d-excess between -50 and 50
ds_nudg['PRECT_dxs'] = ds_nudg['PRECT_dxs'].where(ds_nudg.PRECT_dxs > -25, -25)
ds_nudg['PRECT_dxs'] = ds_nudg['PRECT_dxs'].where(ds_nudg.PRECT_dxs < 50, 50)

# take seasonally weighted averages
#---------------------------------
# shift to get correct months! (UGHHHHH)
# for some reason, xarray reads in the end of the averaging time rather
# than the midpoint of the averaged month (UGH)
# fix from here: https://bb.cgd.ucar.edu/cesm/threads/external-tools-dont-like-cesm-time-coordinate.4604/
time_index_shifted = ds_nudg.time.get_index('time') - timedelta(days=1) 
ds_nudg['time'] = time_index_shifted

# calculate weighted mean using code from xarray tutorial:
# http://xarray.pydata.org/en/stable/examples/monthly-means.html
month_length = ds_nudg.time.dt.days_in_month
# Calculate the weights by grouping by 'time.season'.
weights = month_length.groupby('time.season') / month_length.groupby('time.season').sum()

# Test that the sum of the weights for each season is 1.0
np.testing.assert_allclose(weights.groupby('time.season').sum().values, np.ones(4))

# Calculate the weighted average
ds_nudg_weighted = (ds_nudg * weights).groupby('time.season').sum(dim='time')

# write out files.
ds_nudg_weighted.sel(season='JJA').to_netcdf('../proc_data/iCAM6_nudg_JJA.nc')
ds_nudg_weighted.sel(season='DJF').to_netcdf('../proc_data/iCAM6_nudg_DJF.nc')
ds_nudg.mean(dim="time").to_netcdf('../proc_data/iCAM6_nudg_ANN.nc')

del ds_nudg


In [5]:
# various steps to process nudged file for files 03, 04, and 05.
ds = xr.open_dataset("../raw_data/ts_nudg_subset_2x2.nc", decode_timedelta = False).drop('time_bnds')

#-------------------------------
# Apply some important thresholds.
# some values get numerically unstable at low PRECT rates,
# so need to threshold values.

# calculate d18O_0, need to filter for things above x mm/day
thres = 0.1 # mm/day 
thres_mps = thres*(1/86400)*(1/1000) # convert mm/day -> m/s

ds['PRECT2'] = ds['PRECT'] # keep a copy of PRECT before thresholding for anomaly stuff?

# filter out values below thres_mps?
ds['PRECT'] = ds['PRECT'].where(ds.PRECT > thres_mps, thres_mps)

# d180 between -70 and +10
ds['PRECT_d18O'] = ds['PRECT_d18O'].where(ds.PRECT_d18O > -70, -70)
ds['PRECT_d18O'] = ds['PRECT_d18O'].where(ds.PRECT_d18O < 10, 10)

# constant frac d180 between -70 and +10
ds['PRECT_d18Oec'] = ds['PRECT_d18Oec'].where(ds.PRECT_d18Oec > -70, -70)
ds['PRECT_d18Oec'] = ds['PRECT_d18Oec'].where(ds.PRECT_d18Oec < 10, 10)

# d-excess between -50 and 50
ds['PRECT_dxs'] = ds['PRECT_dxs'].where(ds.PRECT_dxs > -25, -25)
ds['PRECT_dxs'] = ds['PRECT_dxs'].where(ds.PRECT_dxs < 50, 50)

# d180 between -500 and +50
#ds['PRECT_d2H'] = ds['PRECT_d2H'].where(ds.PRECT_d2H > -500, -500)
#ds['PRECT_d2H'] = ds['PRECT_d2H'].where(ds.PRECT_d2H < 50, 50)

# d-excess between -50 and 50
ds['PRECT_dxs'] = ds['PRECT_dxs'].where(ds.PRECT_dxs > -25, -25)
ds['PRECT_dxs'] = ds['PRECT_dxs'].where(ds.PRECT_dxs < 50, 50)

# evap d180 between -70 and +10
ds['PRECTed18O'] = ds['PRECTed18O'].where(ds.PRECTed18O > -70, -70)
ds['PRECTed18O'] = ds['PRECTed18O'].where(ds.PRECTed18O < 40, 40)

# evap d180 between -70 and +10
ds['PRECTed18O'] = ds['PRECTed18O'].where(ds.PRECTed18O > -70, -70)
ds['PRECTed18O'] = ds['PRECTed18O'].where(ds.PRECTed18O < 40, 40)

# PRECTct between 200 and 325K
ds['PRECTct'] = ds['PRECTct'].where(ds.PRECTct > 200, 200)
ds['PRECTct'] = ds['PRECTct'].where(ds.PRECTct < 325, 325)

# PRECTct between 200 and 325K
ds['PRECTtsrf'] = ds['PRECTtsrf'].where(ds.PRECTct > 200, 200)
ds['PRECTtsrf'] = ds['PRECTtsrf'].where(ds.PRECTct < 350, 350)

# PRECTrh between 0 and 100
ds['PRECTrhsrf'] = ds['PRECTrhsrf'].where(ds.PRECTrhsrf > 0, 0)
ds['PRECTrhsrf'] = ds['PRECTrhsrf'].where(ds.PRECTrhsrf < 100, 100)

# PRECTlnf between -6 and 0
ds['PRECTlnf'] = ds['PRECTlnf'].where(ds.PRECTlnf < 0, 0)
ds['PRECTlnf'] = ds['PRECTlnf'].where(ds.PRECTlnf > -6, -6)

# PRECTews between 0 and 25
ds['PRECTews'] = ds['PRECTews'].where(ds.PRECTews < 25, 25)
ds['PRECTews'] = ds['PRECTews'].where(ds.PRECTews > 0, 0)

# PRECTtime between 0 and 60
ds['PRECTtime'] = ds['PRECTtime'].where(ds.PRECTtime < 40, 40)
ds['PRECTtime'] = ds['PRECTtime'].where(ds.PRECTtime > 0, 0)

# PRECTdist between 0 and 100000
ds['PRECTdist'] = ds['PRECTdist'].where(ds.PRECTdist < 100000, 100000)
ds['PRECTdist'] = ds['PRECTdist'].where(ds.PRECTdist > 0, 0)

#-------------------------------------
# Calculate some new variables at monthly resolution:
#-------------------------------------
# add d'18O as well:
ds['log_d18O'] = np.log(ds.PRECT_d18O/1000 + 1) # set threshold.

# calculate liquid water in eq. w/ ed18O.
ds['tlaE'] = 0.35e9/(ds.PRECTtsrf-((100-ds.PRECTrhsrf)/5))**3\
            -1.666e6/(ds.PRECTtsrf-((100-ds.PRECTrhsrf)/5))**2\
            +6.712e3/(ds.PRECTtsrf-((100-ds.PRECTrhsrf)/5))-7.685
ds['alphaE'] = np.exp(ds.tlaE/1000)
ds['d18O_0'] = 1000*(ds.alphaE*(ds.PRECTed18O/1000+1)-1)

# monthly discrimination below evap-corrected d0
ds['D18O'] = (ds.PRECT_d18O-ds.d18O_0)/(1+ds.d18O_0/1000)
ds['log_D18O'] = np.log(ds.D18O/1000 + 1)

ds['qonp'] = (ds.TMQ/1000)/ds.PRECT/86400 # convert to days from seconds

ds['PRECTedxs'] = ds.PRECTed2H - 8*ds.PRECTed18O

ds['dxs_diff'] = (ds.PRECT_dxs - ds.PRECTedxs)

ds['d18O_anom_norm'] = (ds.PRECT_d18O - ds.PRECT_d18O.mean(dim="time"))/ds.PRECT_d18O.mean(dim="time")
ds['RT_anom_norm']   = (ds.PRECTtime - ds.PRECTtime.mean(dim="time"))/ds.PRECTtime.mean(dim="time")
ds['qonp_anom_norm'] = (ds.qonp - ds.qonp.mean(dim="time"))/ds.qonp.mean(dim="time")

# write monthly timeseries.
ds.to_netcdf('../proc_data/iCAM6_nudg_monthly.nc')

#------------------------------------
# get long-term monthly averages.
tmp = ds * ds['PRECT']
monthly_avr = tmp.groupby('time.month').sum(dim='time')/\
                ds['PRECT'].groupby('time.month').sum('time')

monthly_avr.assign_coords({'lat': ds.lat,
              'lon': ds.lon})

monthly_avr.to_netcdf("../proc_data/iCAM6_nudg_MonAvg.nc")

# take seasonally weighted averages
#---------------------------------
# shift to get correct months! (UGHHHHH)
# for some reason, xarray reads in the end of the averaging time rather
# than the midpoint of the averaged month (UGH)
# fix from here: https://bb.cgd.ucar.edu/cesm/threads/external-tools-dont-like-cesm-time-coordinate.4604/
time_index_shifted = ds.time.get_index('time') - timedelta(days=1) 
ds['time'] = time_index_shifted

# calculate weighted mean using code from xarray tutorial:
# http://xarray.pydata.org/en/stable/examples/monthly-means.html
month_length = ds.time.dt.days_in_month
# Calculate the weights by grouping by 'time.season'.
weights = month_length.groupby('time.season') / month_length.groupby('time.season').sum()

# Test that the sum of the weights for each season is 1.0
np.testing.assert_allclose(weights.groupby('time.season').sum().values, np.ones(4))

# Calculate the weighted average
ds_weighted = (ds * weights).groupby('time.season').sum(dim='time')
ds_unweighted = ds.groupby('time.season').mean(dim='time')

ds_weighted.to_netcdf("../proc_data/iCAM6_nudg_WgtSeasAvg.nc")
ds_unweighted.to_netcdf("../proc_data/iCAM6_nudg_UnwgtSeasAvg.nc")
#-------------------
# Calculate annual averages:
# calculate some precip-weighted time means
# uncorrected evaporation d18O
tmp = ds.PRECT * ds.PRECTed18O.where(ds.PRECTed18O > -100, -100)
ed18O_wgt = tmp.sum(dim="time") / ds.PRECT.sum(dim="time")

# mass-weighted precipitation d18O
tmp = ds.PRECT * ds.PRECT_d18O.where(ds.PRECT_d18O > -100, -100)
d18O_wgt = tmp.sum(dim="time") / ds.PRECT.sum(dim="time")

# mass-weighted precipitation d-excess
tmp = ds.PRECT * ds.PRECT_dxs
dxs_wgt = tmp.sum(dim="time") / ds.PRECT.sum(dim="time")

# mass-weighted evap temperature
tmp = ds.PRECT * ds.PRECTtsrf
tsrf_wgt = tmp.sum(dim="time") / ds.PRECT.sum(dim="time")

# mass-weighted evap relative humidity
tmp = ds.PRECT * ds.PRECTrhsrf
rh_wgt = tmp.sum(dim="time") / ds.PRECT.sum(dim="time")

# mass-weighted constant fractionation d18O
tmp = ds.PRECT * ds.PRECT_d18Oec.where(ds.PRECT_d18Oec > -100, -100)
d18Ocf_wgt = tmp.sum(dim="time") / ds.PRECT.sum(dim="time")

# mass-weighted ln(q/q0) = ln(f)
tmp = ds.PRECT * ds.PRECTlnf
lnf_wgt = tmp.sum(dim="time") / ds.PRECT.sum(dim="time")

# mass-weighted condensation temperature
tmp = ds.PRECT * ds.PRECTct
tcond_wgt = tmp.sum(dim="time") / ds.PRECT.sum(dim="time")

# mass-weighted D18O
tmp = ds.PRECT * ds.D18O.where(ds.D18O > -100, -100)
D18O_wgt = tmp.sum(dim="time") / ds.PRECT.sum(dim="time")

# mass-weighted residence time
tmp = ds.PRECT * ds.PRECTtime
time_wgt = tmp.sum(dim="time") / ds.PRECT.sum(dim="time")

# mass-weighted transport distance
tmp = ds.PRECT * ds.PRECTdist
dist_wgt = tmp.sum(dim="time") / ds.PRECT.sum(dim="time")

# mass-weighted evaporation wind speed
tmp = ds.PRECT * ds.PRECTews
ews_wgt = tmp.sum(dim="time") / ds.PRECT.sum(dim="time")

# mass-weighted d-excess diference
tmp = ds.PRECT * ds.dxs_diff
dxsdiff_wgt = tmp.sum(dim='time')/ds.PRECT.sum(dim='time')

tmq_wgt = ds.TMQ.mean(dim="time")/1000 # convert from kg/m2 to m

qonp_wgt = ds.qonp.mean(dim="time")

#------------------------------
# add some derived versions - log of d18O+1, evap-corrected d18O, log(d18O+1)cfe
d18O_log   = np.log(d18O_wgt.where(d18O_wgt > -100, -100)/1000 + 1)
d18Ocf_log = np.log(d18Ocf_wgt.where(d18Ocf_wgt > -100, -100)/1000 + 1)
D18O_log   = np.log(D18O_wgt.where(D18O_wgt > -100, -100)/1000 + 1)
tmq_log    = np.log10(tmq_wgt)
qonp_log   = np.log10(qonp_wgt)
time_log   = np.log10(time_wgt)

# finally, convert these to a new xarray dataset that we can use for plotting.
ds_p1 = xr.Dataset(
    data_vars={'mean_d18O':(('lat','lon'), d18O_wgt.data),
               'log_d18O':(('lat','lon'), d18O_log.data),
               'mean_d18Ocf':(('lat','lon'), d18Ocf_wgt.data),
               'log_d18Ocf':(('lat','lon'), d18Ocf_log.data),
               'mean_lnf': (('lat','lon'), lnf_wgt.data),
               'mean_Tc':  (('lat','lon'), tcond_wgt.data),
               'mean_Te':  (('lat','lon'), tsrf_wgt.data),
               'mean_RH':  (('lat','lon'), rh_wgt.data),
               'evap_d18O':(('lat','lon'), ed18O_wgt.data),
               'mean_D18O':(('lat','lon'), D18O_wgt.data),
               'log_D18O' :(('lat','lon'), D18O_log.data),
               'landfrac' :(('lat','lon'), ds.isel(time=1).LANDFRAC.data),
               'PRECT'    :(('lat','lon'), ds.PRECT.mean(dim='time').data),
               'mean_RT':  (('lat','lon'), time_wgt.data),
               'mean_dist':  (('lat','lon'), dist_wgt.data),
               'log_RT':   (('lat','lon'), time_log.data),
               'mean_TMQ': (('lat','lon'), tmq_wgt.data),
               'log_TMQ':  (('lat','lon'), tmq_log.data),
               'mean_qonp': (('lat','lon'), qonp_wgt.data),
               'log_qonp': (('lat','lon'), qonp_log.data),
               'mean_dxs': (('lat','lon'), dxs_wgt.data),
               'mean_ews': (('lat','lon'), ews_wgt.data),
               'mean_dxsdiff': (('lat','lon'),dxsdiff_wgt.data)
               },
    coords = {'lat': ds.lat.data,
              'lon': ds.lon.data}
    )

ds_p1.to_netcdf('../proc_data/iCAM6_nudg_AnnAvg.nc')

del ds_p1
del ds
del monthly_avr


/Users/rfiorella/opt/anaconda3/envs/xarray/lib/python3.9/site-packages/xarray/core/computation.py:769: RuntimeWarning: overflow encountered in exp
  result_data = func(*input_data)
/Users/rfiorella/opt/anaconda3/envs/xarray/lib/python3.9/site-packages/xarray/core/computation.py:769: RuntimeWarning: divide by zero encountered in log
  result_data = func(*input_data)
/Users/rfiorella/opt/anaconda3/envs/xarray/lib/python3.9/site-packages/xarray/coding/variables.py:173: RuntimeWarning: invalid value encountered in cast
  encoding["_FillValue"] = dtype.type(fv)
/Users/rfiorella/opt/anaconda3/envs/xarray/lib/python3.9/site-packages/xarray/coding/variables.py:180: RuntimeWarning: invalid value encountered in cast
  encoding["missing_value"] = dtype.type(mv)
/var/folders/vz/51v7vlnd3xd6dctwmyrstss00000gn/T/ipykernel_5136/489157480.py:102: SerializationWarning: saving variable TMQ with floating point data as an integer dtype without any _FillValue to use for NaNs
  ds.to_netcdf('../proc_data/iC

In [6]:
# Process iCAM6 - nudged for seasonal averages

ds_icam5 = xr.open_dataset("../raw_data/icam5_19802004.nc", decode_timedelta = False).drop('time_bnds')

# take seasonally weighted averages
#---------------------------------
# shift to get correct months! (UGHHHHH)
# for some reason, xarray reads in the end of the averaging time rather
# than the midpoint of the averaged month (UGH)
# fix from here: https://bb.cgd.ucar.edu/cesm/threads/external-tools-dont-like-cesm-time-coordinate.4604/
time_index_shifted = ds_icam5.time.get_index('time') - timedelta(days=1) 
ds_icam5['time'] = time_index_shifted

# calculate weighted mean using code from xarray tutorial:
# http://xarray.pydata.org/en/stable/examples/monthly-means.html
month_length = ds_icam5.time.dt.days_in_month
# Calculate the weights by grouping by 'time.season'.
weights = month_length.groupby('time.season') / month_length.groupby('time.season').sum()

# Test that the sum of the weights for each season is 1.0
np.testing.assert_allclose(weights.groupby('time.season').sum().values, np.ones(4))

# Calculate the weighted average
ds_icam5_weighted = (ds_icam5 * weights).groupby('time.season').sum(dim='time')

# d180 between -70 and +10
ds_icam5_weighted['PRECT_d18O'] = 1000*(ds_icam5_weighted.PRECT_H218O/ds_icam5_weighted.PRECT_H216O-1)
ds_icam5_weighted['PRECT_d18O'] = ds_icam5_weighted['PRECT_d18O'].where(ds_icam5_weighted.PRECT_d18O > -70, -70)
ds_icam5_weighted['PRECT_d18O'] = ds_icam5_weighted['PRECT_d18O'].where(ds_icam5_weighted.PRECT_d18O < 10, 10)

# d2H between -600 and +50
ds_icam5_weighted['PRECT_d2H'] = 1000*(ds_icam5_weighted.PRECT_HDO/ds_icam5_weighted.PRECT_H216O-1)
ds_icam5_weighted['PRECT_d2H'] = ds_icam5_weighted['PRECT_d2H'].where(ds_icam5_weighted.PRECT_d2H > -600, -600)
ds_icam5_weighted['PRECT_d2H'] = ds_icam5_weighted['PRECT_d2H'].where(ds_icam5_weighted.PRECT_d2H < 80, 80)

# d180 between -70 and +10
ds_icam5_weighted['PRECT_dxs'] = ds_icam5_weighted.PRECT_d2H - 8*ds_icam5_weighted.PRECT_d18O
ds_icam5_weighted['PRECT_dxs'] = ds_icam5_weighted['PRECT_dxs'].where(ds_icam5_weighted.PRECT_dxs > -50, -50)
ds_icam5_weighted['PRECT_dxs'] = ds_icam5_weighted['PRECT_dxs'].where(ds_icam5_weighted.PRECT_dxs < 50, 50)

ds_icam5_weighted

# write out files.
ds_icam5_weighted.sel(season='JJA').to_netcdf('../proc_data/iCAM5_free_JJA.nc')
ds_icam5_weighted.sel(season='DJF').to_netcdf('../proc_data/iCAM5_free_DJF.nc')
ds_icam5_weighted.mean(dim="season").to_netcdf('../proc_data/iCAM5_free_ANN.nc')